In [70]:
import pandas as pd

df = pd.read_csv('/Users/giovanni-lunetta/stat_3255/stat_3255_final/data/cleaned.csv')

In [71]:
df = df.dropna()

In [72]:
top20complaints = df['Complaint Type'].value_counts().nlargest(20).index.tolist()

In [73]:
df['top20complaint'] = 'other'

In [74]:
df = df.reset_index(drop=True)

In [75]:
for i in range(len(df)):
    if df.loc[i, 'Complaint Type'] in top20complaints:
        df.loc[i, 'top20complaint'] = df.loc[i, 'Complaint Type']

In [76]:
df = df.drop('Complaint Type', axis=1)[['housing_units', 'occupied_housing_units', 'median_home_value', 'population', 'median_household_income', 'population_density', 'top20complaint']]

In [77]:
df.head()

,housing_units,occupied_housing_units,median_home_value,population,median_household_income,population_density,top20complaint
0,11195.0,10645.0,574200.0,28606.0,75335.0,16269.0,illegal parking
1,24078.0,22554.0,427100.0,66631.0,57776.0,26903.0,general construction/plumbing
2,37251.0,34843.0,495800.0,98592.0,50799.0,27290.0,street condition
3,12975.0,12227.0,406200.0,38912.0,78667.0,12553.0,missed collection
4,34028.0,30859.0,598000.0,86408.0,42170.0,46966.0,illegal parking


In [78]:
from sklearn.preprocessing import StandardScaler

# Create a StandardScaler object
scaler = StandardScaler()

# select the variables to scale
variables_to_scale = ['housing_units', 'occupied_housing_units', 'median_home_value', 'population', 'median_household_income', 'population_density']

# apply the scaler to the selected variables
df.loc[:, variables_to_scale] = scaler.fit_transform(df.loc[:, variables_to_scale])

In [79]:
df['top20complaint'].value_counts()

noise                            36881
illegal parking                  34172
other                            32376
heat/hot water                   29661
blocked driveway                 12629
unsanitary condition              6686
abandoned vehicle                 4981
plumbing                          4641
missed collection                 4040
paint/plaster                     3760
derelict vehicles                 3571
water system                      3495
dirty condition                   3125
door/window                       2805
rodent                            2778
general construction/plumbing     2582
street condition                  2444
water leak                        2417
illegal dumping                   2034
electric                          1850
sewer                             1810
Name: top20complaint, dtype: int64

In [80]:
# Create a dictionary to map complaint types to numerical values
complaint_map = {'other': 0}
for i, complaint in enumerate(top20complaints, 1):
    complaint_map[complaint] = i

# Replace the complaint types with numerical values
df['complaint_code'] = df['top20complaint'].replace(complaint_map)

In [81]:
df.drop(['top20complaint'], axis=1, inplace=True)

In [82]:
# from sklearn.model_selection import train_test_split

# # Split the data into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(df.drop('complaint_code', axis=1), df['complaint_code'], test_size=0.2, random_state=42)

# # Split the training data into training and validation sets
# X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# Split the cleaned data into train and validation sets
X_train, X_val, y_train, y_val = train_test_split(df.drop(['over3h'], axis=1), df['over3h'], test_size=0.2, random_state=42)
X_test = df_test.drop(['over3h'], axis=1)
y_test = df_test['over3h']

# Align the test data with the training data to add any missing columns
X_test_aligned, X_train_aligned = X_test.align(X_train, join='outer', axis=1, fill_value=0)
X_test = X_test_aligned.loc[:, X_train.columns]

In [83]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import plot_model
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras import regularizers
from sklearn.metrics import f1_score
import os
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE

# Define model architecture
model = tf.keras.Sequential([
    tf.keras.layers.Dense(units=10, activation='relu', kernel_regularizer=regularizers.l2(0.01), input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(units=5, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    tf.keras.layers.Dense(units=21, activation='softmax')
])

# Define optimizer with learning rate
optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)

# Compile model with optimizer
model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# Train model
history = model.fit(X_train, y_train, epochs=25, batch_size=32, validation_data=(X_val, y_val))

Epoch 1/25
3975/3975 [==============================] - 6s 1ms/step - loss: 2.3953 - accuracy: 0.2322 - val_loss: 2.3765 - val_accuracy: 0.2339
Epoch 2/25
3975/3975 [==============================] - 5s 1ms/step - loss: 2.3765 - accuracy: 0.2383 - val_loss: 2.3753 - val_accuracy: 0.2283
Epoch 3/25
3975/3975 [==============================] - 4s 1ms/step - loss: 2.3724 - accuracy: 0.2453 - val_loss: 2.3618 - val_accuracy: 0.2427
Epoch 4/25
3975/3975 [==============================] - 5s 1ms/step - loss: 2.3696 - accuracy: 0.2445 - val_loss: 2.3658 - val_accuracy: 0.2356
Epoch 5/25
3975/3975 [==============================] - 5s 1ms/step - loss: 2.3695 - accuracy: 0.2440 - val_loss: 2.3623 - val_accuracy: 0.2401
Epoch 6/25
3975/3975 [==============================] - 5s 1ms/step - loss: 2.3689 - accuracy: 0.2438 - val_loss: 2.3584 - val_accuracy: 0.2532
Epoch 7/25
3975/3975 [==============================] - 5s 1ms/step - loss: 2.3686 - accuracy: 0.2448 - val_loss: 2.3646 - val_accuracy: